### Data cleaning in Python

This notebook provides examples of:

* reading csv files
* removing columns
* dealing with NAs
* checking and changing data types
* factor encoding and one-hot encoding



In [1]:
### load the data
import pandas as pd

df = pd.read_csv('diabetes.csv')
print(df.head())
print('\nDimensions of data frame:', df.shape)

   row  pregnant  glucose  pressure  triceps  insulin  mass  pedigree  age  \
0    1         6    148.0      72.0     35.0      NaN  33.6     0.627   50   
1    2         1     85.0      66.0     29.0      NaN  26.6     0.351   31   
2    3         8    183.0      64.0      NaN      NaN  23.3     0.672   32   
3    4         1     89.0      66.0     23.0     94.0  28.1     0.167   21   
4    5         0    137.0      40.0     35.0    168.0  43.1     2.288   33   

  diabetes  
0      pos  
1      neg  
2      pos  
3      neg  
4      pos  

Dimensions of data frame: (768, 10)


In [2]:
# remove "row" and "pedigree" columns

df = df.drop(columns=['row', 'pedigree'])
print(df.head())

   pregnant  glucose  pressure  triceps  insulin  mass  age diabetes
0         6    148.0      72.0     35.0      NaN  33.6   50      pos
1         1     85.0      66.0     29.0      NaN  26.6   31      neg
2         8    183.0      64.0      NaN      NaN  23.3   32      pos
3         1     89.0      66.0     23.0     94.0  28.1   21      neg
4         0    137.0      40.0     35.0    168.0  43.1   33      pos


In [3]:
# check for NAs

df.isnull().sum()

pregnant      0
glucose       5
pressure     35
triceps     227
insulin     374
mass         11
age           0
diabetes      0
dtype: int64

In [4]:
# fill triceps and insulin with average
import numpy as np

tri_mean = np.mean(df.triceps)
df.triceps.fillna(tri_mean, inplace=True)

insulin_mean = np.mean(df.insulin)
df.insulin.fillna(insulin_mean, inplace=True)

df.isnull().sum()

pregnant     0
glucose      5
pressure    35
triceps      0
insulin      0
mass        11
age          0
diabetes     0
dtype: int64

In [5]:
# drop remaining rows

df = df.dropna()
print('\nDimensions of data frame:', df.shape)


Dimensions of data frame: (724, 8)


In [6]:
# check data types

df.dtypes

pregnant      int64
glucose     float64
pressure    float64
triceps     float64
insulin     float64
mass        float64
age           int64
diabetes     object
dtype: object

In [7]:
# convert diabetes to categorical data type

df1 = df.copy()

df1.diabetes = df1.diabetes.astype('category')

print(df1.dtypes, "\n")
print(df1.head())     

pregnant       int64
glucose      float64
pressure     float64
triceps      float64
insulin      float64
mass         float64
age            int64
diabetes    category
dtype: object 

   pregnant  glucose  pressure   triceps     insulin  mass  age diabetes
0         6    148.0      72.0  35.00000  155.548223  33.6   50      pos
1         1     85.0      66.0  29.00000  155.548223  26.6   31      neg
2         8    183.0      64.0  29.15342  155.548223  23.3   32      pos
3         1     89.0      66.0  23.00000   94.000000  28.1   21      neg
4         0    137.0      40.0  35.00000  168.000000  43.1   33      pos


In [8]:
# convert diabetes to categorical data type with numeric factor codes

df2 = df.copy()

df2.diabetes = df1.diabetes.astype('category').cat.codes

print(df2.dtypes, "\n")
print(df2.head())  

pregnant      int64
glucose     float64
pressure    float64
triceps     float64
insulin     float64
mass        float64
age           int64
diabetes       int8
dtype: object 

   pregnant  glucose  pressure   triceps     insulin  mass  age  diabetes
0         6    148.0      72.0  35.00000  155.548223  33.6   50         1
1         1     85.0      66.0  29.00000  155.548223  26.6   31         0
2         8    183.0      64.0  29.15342  155.548223  23.3   32         1
3         1     89.0      66.0  23.00000   94.000000  28.1   21         0
4         0    137.0      40.0  35.00000  168.000000  43.1   33         1


The diabetes data is ready to go. 

Two ways of encoding categorical data were shown above. The sklearn algorithms handle either type, but the numeric encoding is more commonly used.

### One-hot encoding

Some algorithms can also accept one-hot encoding. As an example of one-hot encoding, we will use a subset of the diabetes data and the number of pregnancies column.

In [9]:
df3 = df.copy()

df3 = df3[df3.pregnant < 5]
print(df3.shape)

(463, 8)


In [10]:
print(df3.head())

   pregnant  glucose  pressure  triceps     insulin  mass  age diabetes
1         1     85.0      66.0     29.0  155.548223  26.6   31      neg
3         1     89.0      66.0     23.0   94.000000  28.1   21      neg
4         0    137.0      40.0     35.0  168.000000  43.1   33      pos
6         3     78.0      50.0     32.0   88.000000  31.0   26      pos
8         2    197.0      70.0     45.0  543.000000  30.5   53      pos


In [11]:
ohe = pd.get_dummies(df3.pregnant)
print(ohe[:5])

   0  1  2  3  4
1  0  1  0  0  0
3  0  1  0  0  0
4  1  0  0  0  0
6  0  0  0  1  0
8  0  0  1  0  0


The ohe (one-hot-encoded) data above has one column for each possible value 0 - 4. Each row will have only one "1" for each row, with the rest 0.

Now we need to concatenate this with the other columns. The example uses a subset of the columns

In [43]:
df_ohe = pd.concat([ohe, df3.loc[:, df3.columns != 'pregnant']], axis=1)
df_ohe.head()

,0,1,2,3,4,glucose,pressure,triceps,insulin,mass,age,diabetes
1,0,1,0,0,0,85.0,66.0,29.0,155.548223,26.6,31,neg
3,0,1,0,0,0,89.0,66.0,23.0,94.000000,28.1,21,neg
4,1,0,0,0,0,137.0,40.0,35.0,168.000000,43.1,33,pos
6,0,0,0,1,0,78.0,50.0,32.0,88.000000,31.0,26,pos
8,0,0,1,0,0,197.0,70.0,45.0,543.000000,30.5,53,pos
